In [1]:
import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')

In [3]:
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [4]:
bc = nx.betweenness_centrality(g)

In [5]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'betweenness': [i for i in bc.values()]
})

In [6]:
char_infos = pd.read_csv('dataset/charcters_stats.csv', index_col=0)

In [7]:
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [8]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [9]:
# semi_df = semi_df[semi_df['ConnAmnt'] > 30]
semi_df

,id,Name,ConnAmnt,betweenness,Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total
0,1009220,Captain America,565.0,3.536019e-02,good,63,19,35,56,46,100,319
1,1010740,Winter Soldier,229.0,3.022044e-03,good,56,32,35,65,60,84,332
2,1009471,Nick Fury,411.0,1.619165e-02,good,75,11,23,42,25,100,276
3,1009515,Punisher,276.0,1.125980e-02,good,50,16,23,28,22,100,239
4,1009535,Red Skull,237.0,2.700376e-03,bad,75,10,12,14,19,80,210
...,...,...,...,...,...,...,...,...,...,...,...,...
192,1011092,Leech,45.0,4.212098e-05,good,25,5,12,14,62,14,132
193,1011071,Demogoblin,14.0,1.922775e-06,bad,50,48,42,35,54,60,289
194,1009248,Fabian Cortez,19.0,8.025296e-07,bad,1,1,1,1,0,1,5
195,1011073,Bloodaxe,11.0,7.964722e-06,bad,63,80,33,80,84,84,424


In [10]:
# Por enquanto não estamos usando os poderes
# powers = pd.read_csv('dataset/superheroes_power_matrix.csv')
# complete_df = pd.merge(semi_df, powers, how='inner', on=['Name'])
# complete_df

In [11]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [12]:
import statsmodels.api as sm

model = sm.OLS(semi_df['betweenness'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            betweenness   R-squared (uncentered):                   0.212
Model:                            OLS   Adj. R-squared (uncentered):              0.200
Method:                 Least Squares   F-statistic:                              17.06
Date:                Wed, 11 Nov 2020   Prob (F-statistic):                    7.49e-10
Time:                        14:27:18   Log-Likelihood:                          656.48
No. Observations:                 193   AIC:                                     -1307.
Df Residuals:                     190   BIC:                                     -1297.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence  8.662e-05   2.58e-05      3.363      0.001    3.58e-05       0.000
Strength      6.631e-06   2.11e-05      0.315      0.753   -3.49e-05    4.82e-05
Power        -2.653e-05   2.69e-05     -0.985      0.326   -7.96e-05    2.66e-05
==============================================================================
Omnibus:                      235.851   Durbin-Watson:                   0.830
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9262.677
Skew:                           5.105   Prob(JB):                         0.00
Kurtosis:                      35.366   Cond. No.                         5.65
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""